In [1]:
import io
import pandas as pd
import re
import requests
from typing import *
import sys

sys.path.insert(0, "..")
from helpers import add_to_dict, log
from helpers_csv import csvs_to_dict, dict_to_csvs

In [2]:
csv = requests.get(
    "https://www.data.gv.at/katalog/dataset/15d6ede8-f128-4fcd-aa3a-4479e828f477/resource/804f6db1-add7-4480-b4d0-e52e61c48534/download/worttabelle.csv"
).content
text = re.sub(";;\r\n", "\n", csv.decode("utf-8"))
df = pd.read_csv(io.StringIO(text))
df.head()

,Laenge,Hauptwort,Vorschlag,Binnen
0,50,Verantwortlicher für Informationssicherheit (C...,CISO,N
1,50,Verantwortlicher für Informationssicherheit (C...,Verantwortliche bzw. Verantwortlicher für Info...,N
2,45,Diplomierte Gesundheits- und Krankenschwester,Diplomiertes Krankenpflegepersonal,N
3,43,Unabhängiger Bedienstetenschutzbeauftragter,Unabhängige Bedienstetenschutzbeauftragte bzw....,N
4,39,Kontrakt- und Berichtswesenbeauftragter,Kontrakt- und Berichtswesenbeauftragte bzw. -b...,N


In [3]:
df.to_csv(
    "vienna_catalog_raw.csv",
    index=False,
)

We change Binnen-I to gender star to have one simple style, and we try to attribute singular and plural as well as possible:

In [4]:
dic: Dict[str, Dict[str, List[str]]] = {"sg": {}, "pl": {}}
for (_, _, ungendered, gendered, binnenI) in df.to_records():
    if binnenI == "Y":
        gendered = re.sub(r"([a-zäöüß])I", r"\1*i", gendered)
    if type(gendered) == str:  # skip ill-formatted rows
        if (
            re.findall(r"[iI]n( .*)?$", gendered) != []
            or re.findall(r" bzw\.? ", gendered) != []
        ):
            add_to_dict(ungendered, [gendered], dic["sg"])
        elif (
            re.findall(r"[iI]nnen( .*)?$", gendered) != []
            or re.findall(r" und ", gendered) != []
        ):
            add_to_dict(ungendered, [gendered], dic["pl"])
        else:
            add_to_dict(ungendered, [gendered], dic["sg"])
            add_to_dict(ungendered, [gendered], dic["pl"])

In [5]:
dict_to_csvs(dic, "vienna_catalog")

We can read this CSV back to a Python dictionary with the following method:

In [6]:
dic = csvs_to_dict("vienna_catalog")
list(dic["sg"].items())[:5]

[]